In [1]:
# install
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
# importing libraries
import os
import tempfile
import pprint

from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
# loading datasets
# since the MovieLens dataset does not have predefined splits, all data are under train split
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [4]:
# printing single movie
for i in movies.take(1).as_numpy_iterator():
  pprint.pprint(i)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [5]:
# printing single rating
for i in ratings.take(1).as_numpy_iterator():
  pprint.pprint(i)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
movies = movies.map(lambda x:x["movie_title"])

In [7]:
list(movies)[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'You So Crazy (1994)'>

In [8]:
# selecting the basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

In [9]:
list(ratings)[0]

{'movie_title': <tf.Tensor: shape=(), dtype=string, numpy=b"One Flew Over the Cuckoo's Nest (1975)">,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'138'>}

In [10]:
# spliting data into training and evaluation set
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=True)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [11]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(100_000).map(lambda x: x['user_id'])

In [12]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [13]:
print(f'unique_movie_titles: {unique_movie_titles[:5]}')
print(f'unique_user_ids: {unique_user_ids[:5]}')

unique_movie_titles: [b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)']
unique_user_ids: [b'1' b'10' b'100' b'101' b'102']


In [14]:
embedding_dimension = 32 # increase this for more complexity

In [15]:
# query tower
user_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None
    ),
    # we add additional embedding to account for unknown tokens
    tf.keras.layers.Embedding(len(unique_user_ids)+1, embedding_dimension)
])

In [16]:
# candidate tower
movie_model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None
    ),
    tf.keras.layers.Embedding(len(unique_movie_titles)+1, embedding_dimension)
])

In [17]:
# metrics
metrics = tfrs.metrics.FactorizedTopK(
    candidates = movies.batch(128).map(movie_model)
)

In [18]:
metrics

In [19]:
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

In [20]:
# full model
class MovielensModel(tfrs.Model):
  def __init__(self, user_model, movie_model):
    super().__init__()
    self.user_model: tf.keras.Model = user_model
    self.movie_model: tf.keras.Model = movie_model
    self.task: tf.keras.layers.Layer = task
  
  def compute_loss(self, features: Dict[Text, tf.Tensor] , training=False) -> tf.Tensor:
    user_embedding = self.user_model(features["user_id"])
    positive_movie_embedding = self.movie_model(features["movie_title"])
    return self.task(user_embedding, positive_movie_embedding)

In [22]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [23]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [24]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 28s 2s/step - factorized_top_k/top_1_categorical_accuracy: 3.2500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0040 - factorized_top_k/top_10_categorical_accuracy: 0.0107 - factorized_top_k/top_50_categorical_accuracy: 0.0770 - factorized_top_k/top_100_categorical_accuracy: 0.1516 - loss: 69896.7550 - regularization_loss: 0.0000e+00 - total_loss: 69896.7550
Epoch 2/3
10/10 [==============================] - 26s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0024 - factorized_top_k/top_5_categorical_accuracy: 0.0163 - factorized_top_k/top_10_categorical_accuracy: 0.0342 - factorized_top_k/top_50_categorical_accuracy: 0.1594 - factorized_top_k/top_100_categorical_accuracy: 0.2831 - loss: 67562.6705 - regularization_loss: 0.0000e+00 - total_loss: 67562.6705
Epoch 3/3
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0030 - factorized_top_k/top_5_categorical_accuracy: 0.

In [25]:
model.evaluate(cached_test, return_dict=True) # performance need to be improved

5/5 [==============================] - 8s 920ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0046 - factorized_top_k/top_5_categorical_accuracy: 0.0270 - factorized_top_k/top_10_categorical_accuracy: 0.0526 - factorized_top_k/top_50_categorical_accuracy: 0.2050 - factorized_top_k/top_100_categorical_accuracy: 0.3334 - loss: 30292.2142 - regularization_loss: 0.0000e+00 - total_loss: 30292.2142


{'factorized_top_k/top_100_categorical_accuracy': 0.3333500027656555,
 'factorized_top_k/top_10_categorical_accuracy': 0.05260000005364418,
 'factorized_top_k/top_1_categorical_accuracy': 0.004550000187009573,
 'factorized_top_k/top_50_categorical_accuracy': 0.2049500048160553,
 'factorized_top_k/top_5_categorical_accuracy': 0.027049999684095383,
 'loss': 27503.27734375,
 'regularization_loss': 0,
 'total_loss': 27503.27734375}

In [27]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

index.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

In [28]:
_, titles = index(tf.constant(["42"]))
print(f'recommendation for user with id 42 is {titles[0,:3]}')

recommendation for user with id 42 is [b"Kid in King Arthur's Court, A (1995)" b'Rudy (1993)'
 b'Old Yeller (1957)']
